In [3]:
# IMPORT MODELS

# to import models you need to add src path to sys.path

import os
import sys
cwd = os.getcwd()
# get src path
src_path = cwd.rsplit("/", 3)[0]

# add it to sys.path
sys.path.append(src_path)

# then import data from `process_data` directory
from process_data.models import ProjectItem, VoterItem

In [4]:
import csv
import collections
import os
import ast

In [5]:
path = os.getcwd()
csv_files_path = path.split("/")[:-3]
csv_files_path = os.path.join("/", *path.split("/")[:-3], "data", "Stanford", "2023")

In [6]:
# elections
elections = f'{csv_files_path}/elections_rich.csv'

# projects
projects = f'{csv_files_path}/projects_clean.csv'

# votes
votes = {
    "vote_approvals": f'{csv_files_path}/vote_approvals_clean.csv',
    "vote_infer_knapsack_partial": f'{csv_files_path}/vote_infer_knapsack_partial.csv',
    "vote_infer_knapsack_skip": f'{csv_files_path}/vote_infer_knapsack_skip.csv',
    "vote_knapsacks_clean": f'{csv_files_path}/vote_knapsacks_clean.csv',
    "vote_rankings_clean": f'{csv_files_path}/vote_rankings_clean.csv',
    "vote_tokens_clean": f'{csv_files_path}/vote_tokens_clean.csv'
}

# voters
voters = f'{csv_files_path}/voters_clean.csv'

In [7]:
elections_reader = csv.DictReader(open(elections))
elections_dict = {row["election_id"]: row for row in elections_reader}

In [8]:
voters_per_election = {}
voters_reader = csv.DictReader(open(voters))
for row in voters_reader:
    election_id = str(row["election_id"])
    if not voters_per_election.get(election_id):
         voters_per_election[election_id] = collections.defaultdict(list)
    voters_per_election[election_id][row["voter_id"]].append(row)

In [ ]:
# votes_per_election = {}
# vote_approvals_reader = csv.DictReader(open(vote_approvals))
# for row in vote_approvals_reader:
#     if not votes_per_election.get(row["election_id"]):
#          votes_per_election[row["election_id"]] = collections.defaultdict(list)
#     votes_per_election[row["election_id"]][row["voter_id"]].append(row["project_id"])

In [9]:
all_votes = {}
for vote_name, vote_path in votes.items():
     vote = {}
     vote_reader = csv.DictReader(open(vote_path))
     for row in vote_reader:
         if not vote.get(row["election_id"]):
              vote[row["election_id"]] = collections.defaultdict(list)
         vote[row["election_id"]][row["voter_id"]].append(row["project_id"])
     all_votes[vote_name] = vote


In [10]:
all_votes["vote_approvals"]["3"]["3-0"]

['23', '28', '27', '29']

In [11]:
projects_per_election = collections.defaultdict(list)
projects_reader = csv.DictReader(open(projects))
for row in projects_reader:
    projects_per_election[row["election_id"]].append(row)

In [12]:
election = elections_dict['3']
print(election['name'])
print(election['budget'])
print(election['max_n_projects'])
# real election: binary indicator whether the election organizer indicated that real decisions were based on this exercise.
print(election['real_election'])
print(election['phase_0'])
# voters total: number of voters that cast any ballot in this election.
print(election['voters_total'])
print(election['total_n_projects'])
# – setting off approval k projects, setting off approval k projects,
# setting off ranking k projects, setting exp approval k projects,
# textitsetting exp ranking k projects: Number of projects that the
# voter can select in the primary (’off’) or secondary (’exp’) approval/ranking
# ballot.

PB Long Beach District 9 2015
250000.0
4.0
1
approval
231
11


In [13]:
voters = voters_per_election['166']
print(voters['166-0'])
# for voter, voter_data in voters.items():
#     print(voter, voter_data[0]["authentication_method"])
    # raise RuntimeError

[{'': '77666', 'election_id': '166', 'authentication_method': 'phone', 'stage': 'done', 'void': '0', 'updated_at': '2021-01-09 ', 'voter_id': '166-0'}]


In [14]:
def create_project_items(projects):
    all_projects = []
    for project in projects:
        project_id = project["project_id"]
        item = ProjectItem(project_id)
        category = project.get("category_id")
        if category:
            item.category = int(category.replace(".0", ""))
        item.cost = project["cost"]
        map_geometry = project.get("map_geometry")
        if map_geometry:
            map_geometry = ast.literal_eval(map_geometry)
            if isinstance(map_geometry[0], dict):
                map_geometry = map_geometry[0]["coordinates"]
            coordinates = [float(sum(col))/len(col) for col in zip(*map_geometry)]

            item.latitude, item.longitude = coordinates
        all_projects.append(item)
    return all_projects

In [22]:
for election_id, election_data in elections_dict.items():
    print(election_data)
    print(election_data["name"])
    raise RuntimeError

{'': '0', 'election_id': '3', 'name': 'PB Long Beach District 9 2015', 'budget': '250000.0', 'max_n_projects': '4.0', 'ballot_text': '1811.0', 'time_zone': 'Pacific Time (US & Canada)', 'real_election': '1', 'phase_0': 'approval', 'phase_1': 'thanks_approval', 'phase_2': 'comparison', 'phase_3': 'survey', 'phase_4': 'thanks', 'phase_5': 'done', 'phase_6': '', 'phase_7': '', 'phase_8': '', 'voters_total': '231', 'total_n_projects': '11', 'setting_off_approval_k_projects': '4.0', 'setting_off_ranking_k_projects': '', 'setting_exp_approval_k_projects': '', 'setting_exp_ranking_k_projects': '', 'voters_off_approval': '', 'voters_off_knapsack': '', 'voters_off_ranking': '', 'voters_off_token': '', 'voters_off_bftb': '', 'voters_off_plusminus': '', 'voters_off_comparison': '', 'voters_exp_approval': '', 'voters_exp_knapsack': '', 'voters_exp_ranking': '', 'voters_exp_token': '', 'voters_exp_bftb': '', 'voters_exp_plusminus': '', 'voters_exp_comparison': '', 'voters_off_thanks': '', 'voters_s

RuntimeError: 

In [20]:
for election_id, election_data in elections_dict.items():
    print(election_data)
    
    max_length = election_data.get("max_n_projects") or election_data.get("setting_off_approval_k_projects")
    budget = election_data["budget"]
    name = election_data["name"]
    is_real_election = election_data["real_election"]
    voters_total = election_data["voters_total"]
    projects_total = election_data["total_n_projects"]
    print(f"max_length: {max_length}")
    
    election_data.get
    voters = voters_per_election[election_id]
    projects = projects_per_election[election_id]
    all_projects = create_project_items(projects)
    for vote_name, vote_data in all_votes.items():
        if vote_data.get(election_id):
            print(vote_name)
            all_voters = []
            for voter_id, votes in vote_data[election_id].items():
                voter = voters[voter_id]
                if voter[0]["void"] != "0":
                    raise RuntimeError(
                        f"Voter `{voter_id}` has a void vote! "
                        f"Should be removed! Election ID: {election_id}")
                voter_item = VoterItem(voter_id)
                voter_item.vote = ','.join(votes)
                if vote_name == "vote_approvals":
                    # "vote_infer_knapsack_partial"
                    # "vote_infer_knapsack_skip"
                    # "vote_knapsacks_clean"
                    # "vote_rankings_clean"
                    # "vote_tokens_clean"
                    all_voters.append(voter_item)
                # elif vote_name == ""
            print(all_voters)
            raise RuntimeError

{'': '0', 'election_id': '3', 'name': 'PB Long Beach District 9 2015', 'budget': '250000.0', 'max_n_projects': '4.0', 'ballot_text': '1811.0', 'time_zone': 'Pacific Time (US & Canada)', 'real_election': '1', 'phase_0': 'approval', 'phase_1': 'thanks_approval', 'phase_2': 'comparison', 'phase_3': 'survey', 'phase_4': 'thanks', 'phase_5': 'done', 'phase_6': '', 'phase_7': '', 'phase_8': '', 'voters_total': '231', 'total_n_projects': '11', 'setting_off_approval_k_projects': '4.0', 'setting_off_ranking_k_projects': '', 'setting_exp_approval_k_projects': '', 'setting_exp_ranking_k_projects': '', 'voters_off_approval': '', 'voters_off_knapsack': '', 'voters_off_ranking': '', 'voters_off_token': '', 'voters_off_bftb': '', 'voters_off_plusminus': '', 'voters_off_comparison': '', 'voters_exp_approval': '', 'voters_exp_knapsack': '', 'voters_exp_ranking': '', 'voters_exp_token': '', 'voters_exp_bftb': '', 'voters_exp_plusminus': '', 'voters_exp_comparison': '', 'voters_off_thanks': '', 'voters_s

RuntimeError: 